In [2]:
import findspark
findspark.init('/opt/spark')  # Adjust the path if Spark is installed elsewhere

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("example").getOrCreate()

24/05/21 21:10:42 WARN Utils: Your hostname, chuot-HP-Pavilion-Laptop-14-ce3xxx resolves to a loopback address: 127.0.1.1; using 192.168.1.5 instead (on interface wlo1)
24/05/21 21:10:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/21 21:10:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/05/21 21:10:49 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [69]:
# Define the path to your JSON file
from pyspark.sql.functions import col, udf, to_date, when, regexp_extract, explode, col, regexp_replace, to_timestamp, concat
from pyspark.sql.types import StructType, StructField, IntegerType, FloatType, StringType
import json
json_file_path = 'vieclam24h.json'
# Read the JSON file into a DataFrame
df = spark.read.option("multiline","true").json(json_file_path)

# df_exploded = df.select(explode(col("category")).alias("unique_category"))

# unique_categories = df_exploded.select("unique_category").distinct().collect()
# print(len(unique_categories))
# for row in unique_categories:
#     pass
#     print(row.unique_category)
# unique_experience_df = df.select("experience").distinct()

# unique_experience_df.show()

# from pyspark.sql.functions import col
unique_values = df.select(col("expiration")).distinct().collect()

# In ra các giá trị duy nhất
for row in unique_values:
    print(row.expiration)





#rdd.toDF().select("salary").show(1)

# Salary


30/07/2024
02/06/2024
19/06/2024
21/05/2024
07/06/2024
16/06/2024
08/06/2024
28/05/2024
16/07/2024
31/07/2024
30/05/2024
27/05/2024
18/06/2024
24/05/2024
30/08/2024
01/06/2024
27/06/2024
09/06/2024
04/06/2024
06/06/2024
20/08/2024
29/05/2024
25/05/2024
05/07/2024
31/05/2024
26/05/2024
15/07/2024
15/06/2024
12/06/2024
22/06/2024
03/06/2024
13/06/2024
29/06/2024
17/06/2024
21/06/2024
24/06/2024
23/05/2024
10/06/2024
05/06/2024
11/06/2024
30/06/2024
20/06/2024
14/06/2024
22/05/2024


In [ ]:
# Salary
def parse_salary(salary):
    if salary is None or salary.lower() == "thỏa thuận":
        return json.dumps({"type": 2, "min": None, "max": None, "fixed_value": None})
    elif "triệu" in salary:
        salary = salary.replace(" triệu", "").replace(",", ".").strip()
        if "-" in salary:
            min_salary, max_salary = salary.split("-")
            return json.dumps({"type": 1, "min": float(min_salary), "max": float(max_salary), "fixed_value": None})
        else:
            return json.dumps({"type": 0, "min": None, "max": None, "fixed_value": float(salary)})
    return json.dumps({"type": 2, "min": None, "max": None, "fixed_value": None})

parse_salary_udf = udf(parse_salary, StringType())
df = df.withColumn("salary", parse_salary_udf(col("salary")))
rdd = df.rdd.map(lambda row: row.asDict())

In [34]:
# Experience
def transform_experience(df):
    df = df.withColumn("experience", when(col("experience") == "Chưa có kinh nghiệm", 0).otherwise(col("experience")))
    
    df = df.withColumn("experience", regexp_replace(col("experience"), "(Dưới |Hơn |năm)", ""))
    return df

transformed_df = transform_experience(df)

# Hiển thị kết quả
transformed_df.select("experience").show()

+----------+
|experience|
+----------+
|        2 |
|        1 |
|        1 |
|        1 |
|         0|
|         0|
|        2 |
|        1 |
|        1 |
|         0|
|         0|
|        1 |
|        1 |
|         0|
|         0|
|        1 |
|        1 |
|         0|
|        2 |
|         0|
+----------+
only showing top 20 rows



In [64]:
# Type
# 0. Toàn thời gian cố định
# 1. Bán thời gian cố định
# 2. Toàn thời gian tạm thời
# 3. Khác
df = df.withColumn(
    "type",
     when(col("type") == "Toàn thời gian cố định", 0)
    .when(col("type") == "Bán thời gian cố định", 1)
    .when(col("type") == "Toàn thời gian tạm thời", 2)
    .when(col("type") == "Khác", 3)
    .otherwise(None)
)
df.select("type").show(5)


+----+
|type|
+----+
|   0|
|   0|
|   0|
|   0|
|   0|
+----+
only showing top 5 rows



In [70]:
# Expiration
df = df.withColumn("expiration", to_timestamp(concat(col("expiration"), lit(" 00:00:00")), "dd/MM/yyyy HH:mm:ss"))
df.select("expiration").show(5)

NameError: name 'lit' is not defined